In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from tqdm import tqdm
import wandb

from scipy.signal import find_peaks
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import r2_score
import psutil
import os
import time

In [2]:
wandb.login()
run = wandb.init(project="master-multicomponent-mri", name="single-signal-baseline")

wandb: Currently logged in as: tr-phan (trphan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
class T1Dataset(Dataset):
    def __init__(self, input_data, target_data):
        self.input_data = torch.FloatTensor(input_data.T)  # Shape: (400000, 8)
        self.target_data = torch.FloatTensor(target_data.T)  # Shape: (400000, 32)
    
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        return self.input_data[idx], self.target_data[idx]

In [ ]:
class T1ReconstructionNet(nn.Module):
    def __init__(self):
        super(T1ReconstructionNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(8, 128),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 256),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 32)
        )
    
    def forward(self, x):
        return self.network(x)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=15, min_delta=1e-6, path='saved_models/baseline_nn.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.path = path
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)

In [6]:
data = loadmat('../../data/data_T1_Q32_P8_400k.mat')
input_noisy = data['input_noisy']
ref = data['ref']
no_comp = data['no_comp']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    input_noisy.T, ref.T, 
    test_size=0.2, 
    random_state=42
)

# Create datasets
train_dataset = T1Dataset(X_train.T, y_train.T)
val_dataset = T1Dataset(X_val.T, y_val.T)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T1ReconstructionNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    factor=0.5, 
    patience=10,
    verbose=True
)
early_stopping = EarlyStopping(patience=15)
batch_size = 256

d:\Proton Drive\My files\Schule\Master\Masterarbeit\project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
wandb.config.update({
    "learning_rate": 0.001,
    "batch_size": batch_size,
    "epochs": 1,
    "architecture": "SingleSignalMLP",
    "optimizer": "Adam",
    "loss_function": "MSELoss",
    "scheduler": "ReduceLROnPlateau",
    "early_stopping_patience": 15
})

In [10]:
for epoch in range(200):
    # Training phase
    model.train()
    train_loss = 0.0
    progress_bar_train = tqdm(train_loader, desc=f'Epoch [{epoch+1}/200] Training')
    
    for inputs, targets in progress_bar_train:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        train_loss += loss.item()
        progress_bar_train.set_postfix({'loss': loss.item()})

    avg_train_loss = train_loss / len(train_loader)

    # Validation phase
    model.eval()
    val_loss = 0.0
    progress_bar_val = tqdm(val_loader, desc=f'Epoch [{epoch+1}/200] Validation')
    
    with torch.no_grad():
        for inputs, targets in progress_bar_val:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            progress_bar_val.set_postfix({'loss': loss.item()})

    avg_val_loss = val_loss / len(val_loader)

    # Learning rate scheduling
    scheduler.step(avg_val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    # Early stopping
    early_stopping(avg_val_loss, model)

    # Logging
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "learning_rate": current_lr
    })

    print(f'Epoch [{epoch+1}/200], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}, LR: {current_lr}')

    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Epoch [1/200] Validation: 100%|██████████| 313/313 [00:01<00:00, 278.98it/s, loss=0.0024] 


Epoch [1/200], Train Loss: 0.004034, Val Loss: 0.002669, LR: 0.001


Epoch [2/200] Validation: 100%|██████████| 313/313 [00:00<00:00, 343.26it/s, loss=0.00225]


Epoch [2/200], Train Loss: 0.002740, Val Loss: 0.002608, LR: 0.001


Epoch [3/200] Validation: 100%|██████████| 313/313 [00:01<00:00, 299.27it/s, loss=0.00238]


...

EarlyStopping counter: 13 out of 15
Epoch [98/200], Train Loss: 0.002473, Val Loss: 0.002466, LR: 6.25e-05


Epoch [99/200] Validation: 100%|██████████| 313/313 [00:01<00:00, 308.69it/s, loss=0.00217]


EarlyStopping counter: 14 out of 15
Epoch [99/200], Train Loss: 0.002472, Val Loss: 0.002465, LR: 6.25e-05


Epoch [100/200] Validation: 100%|██████████| 313/313 [00:01<00:00, 306.35it/s, loss=0.00217]

EarlyStopping counter: 15 out of 15
Epoch [100/200], Train Loss: 0.002473, Val Loss: 0.002465, LR: 6.25e-05
Early stopping triggered


In [ ]:
class ModelEvaluator:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.metrics = {}

    def calculate_psnr(self, target, prediction):
        mse = np.mean((target - prediction) ** 2)
        if mse == 0:
            return float('inf')
        max_pixel = np.max(target)
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr

    def calculate_peak_metrics(self, target, prediction, prominence=0.1):
        """Calculate peak detection accuracy"""
        peak_metrics = {'true_peaks': 0, 'predicted_peaks': 0, 'matching_peaks': 0}
        
        for i in range(len(target)):
            true_peaks, _ = find_peaks(target[i], prominence=prominence)
            pred_peaks, _ = find_peaks(prediction[i], prominence=prominence)
            
            # Count matching peaks (within ±1 index)
            matches = 0
            for tp in true_peaks:
                for pp in pred_peaks:
                    if abs(tp - pp) <= 1:
                        matches += 1
                        break
            
            peak_metrics['true_peaks'] += len(true_peaks)
            peak_metrics['predicted_peaks'] += len(pred_peaks)
            peak_metrics['matching_peaks'] += matches
        
        return peak_metrics

    def measure_inference_time(self, input_tensor, num_iterations=100):
        self.model.eval()
        start_time = time.time()
        with torch.no_grad():
            for _ in range(num_iterations):
                _ = self.model(input_tensor)
        end_time = time.time()
        return (end_time - start_time) / num_iterations

    def count_parameters(self):
        return sum(p.numel() for p in self.model.parameters() if p.requires_grad)

    def measure_memory_usage(self):
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / 1024 / 1024  # Convert to MB

    def evaluate_model(self, val_loader):
        self.model.eval()
        metrics = {
            'mse': 0.0,
            'mae': 0.0,
            'psnr': 0.0,
            'r2_score': 0.0,
            'peak_accuracy': {},
            'inference_time': 0.0,
            'memory_usage': 0.0,
            'num_parameters': self.count_parameters()
        }
        
        all_targets = []
        all_predictions = []
        
        # Get sample input for inference time measurement
        sample_input, _ = next(iter(val_loader))
        metrics['inference_time'] = self.measure_inference_time(sample_input.to(self.device))
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                # Convert to numpy for metric calculation
                targets_np = targets.cpu().numpy()
                outputs_np = outputs.cpu().numpy()
                
                all_targets.append(targets_np)
                all_predictions.append(outputs_np)
        
        # Concatenate all batches
        all_targets = np.concatenate(all_targets)
        all_predictions = np.concatenate(all_predictions)
        
        # Calculate metrics
        metrics['mse'] = np.mean((all_targets - all_predictions) ** 2)
        metrics['mae'] = np.mean(np.abs(all_targets - all_predictions))
        metrics['psnr'] = self.calculate_psnr(all_targets, all_predictions)
        metrics['r2_score'] = r2_score(all_targets.flatten(), all_predictions.flatten())
        metrics['peak_metrics'] = self.calculate_peak_metrics(all_targets, all_predictions)
        metrics['memory_usage'] = self.measure_memory_usage()
        
        return metrics

evaluator = ModelEvaluator(model, device)

final_metrics = evaluator.evaluate_model(val_loader)

wandb.log({
    "final_mse": final_metrics['mse'],
    "final_mae": final_metrics['mae'],
    "final_psnr": final_metrics['psnr'],
    "final_r2_score": final_metrics['r2_score'],
    "peak_detection_accuracy": final_metrics['peak_metrics']['matching_peaks'] / final_metrics['peak_metrics']['true_peaks'],
    "inference_time_ms": final_metrics['inference_time'] * 1000,
    "memory_usage_mb": final_metrics['memory_usage'],
    "model_parameters": final_metrics['num_parameters']
})

print("\nFinal Model Evaluation:")
print(f"MSE: {final_metrics['mse']:.6f}")
print(f"MAE: {final_metrics['mae']:.6f}")
print(f"PSNR: {final_metrics['psnr']:.2f} dB")
print(f"R² Score: {final_metrics['r2_score']:.4f}")
print(f"Peak Detection Accuracy: {final_metrics['peak_metrics']['matching_peaks'] / final_metrics['peak_metrics']['true_peaks']:.2%}")
print(f"Average Inference Time: {final_metrics['inference_time']*1000:.2f} ms")
print(f"Memory Usage: {final_metrics['memory_usage']:.1f} MB")
print(f"Number of Parameters: {final_metrics['num_parameters']:,}")


Final Model Evaluation:
MSE: 0.002465
MAE: 0.026724
PSNR: 24.57 dB
R² Score: 0.4523
Peak Detection Accuracy: 35.70%
Average Inference Time: 0.87 ms
Memory Usage: 998.0 MB
Number of Parameters: 336,672


In [ ]:
model.eval()

# Visualization function with peak analysis
def visualize_final_results():
    model.eval()
    results_by_peaks = {1: [], 2: [], 3: [], 4: []}
    
    with torch.no_grad():
        for num_peaks in range(1, 5):
            peak_indices = np.where(no_comp.flatten() == num_peaks)[0]
            sample_indices = np.random.choice(peak_indices, min(10, len(peak_indices)), replace=False)
            
            for idx in sample_indices:
                input_signal = torch.FloatTensor(input_noisy[:, idx]).unsqueeze(0).to(device)
                target = ref[:, idx]
                prediction = model(input_signal).cpu().squeeze(0).numpy()
                mse = np.mean((target - prediction)**2)
                results_by_peaks[num_peaks].append(mse)
    
    for num_peaks, mse_values in results_by_peaks.items():
        avg_mse = np.mean(mse_values)
        wandb.log({f"MSE_{num_peaks}_peaks": avg_mse})
        print(f"Average MSE for {num_peaks} peak(s): {avg_mse:.6f}")
        
visualize_final_results()

Average MSE for 1 peak(s): 0.001977
Average MSE for 2 peak(s): 0.001885
Average MSE for 3 peak(s): 0.002719
Average MSE for 4 peak(s): 0.002013


In [13]:
wandb.finish()

MSE_1_peaks,▁
MSE_2_peaks,▁
MSE_3_peaks,▁
MSE_4_peaks,▁
epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
final_mae,▁
final_mse,▁
final_psnr,▁
final_r2_score,▁
inference_time_ms,▁
learning_rate,████████████████▄▄▄▄▄▄▄▄▄▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁
